Spark版本为2.0.0,Python版本为3.5.2,Jupyter notebook server版本为4.2.1

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.

![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)
![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

# **Web Server Log Analysis with Apache Spark**

This lab will demonstrate how easy it is to perform web server log analysis with Apache Spark.

Server log analysis is an ideal use case(**用例**) for Spark.  It's a very large, common data source and contains a rich set of information.  Spark allows you to store your logs in files on disk cheaply, while still providing a quick and simple way to perform data analysis on them.  This homework will show you how to use Apache Spark on real-world text-based production logs and fully harness(**利用**) the power of that data.  Log data comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for <strong>monitoring servers, improving business and customer(业务和客户) intelligence, building recommendation systems, fraud detection(欺诈检测)</strong>, and much more.
## How to complete this lab

This lab is broken up into sections with bite-sized examples for demonstrating Spark functionality for log processing.

It consists of 5 parts:
* *Part 1:* Introduction and Imports
* *Part 2:* Exploratory Data Analysis(**探索性数据分析EDA**)
* *Part 3*: Analysis Walk-Through on the Web Server Log File
* *Part 4*: Analyzing Web Server Log File
* *Part 5*: Exploring 404 Response Codes

Also, at the very bottom:

* *Appendix A*: Submitting Your Exercises to the Autograder

## Part 1: Introduction and Imports

### A note about DataFrame column references

In Python, it's possible to access a DataFrame's columns either by attribute (`df.age`) or by indexing (`df['age']`). Referring to a column by attribute (`df.age`) is very Pandas-like, and it's highly convenient, especially when you're doing interactive data exploration. But it can fail, for reasons that aren't obvious(**但它可能会失败，原因并不明显**). For example:

In [2]:
throwaway_df = sqlContext.createDataFrame(
    [('Anthony', 10), ('Julia', 20), ('Fred', 5)],
    ('name', 'count'))

#This line does not work.Please comment it out later.
#AttributeError: 'function' object has no attribute '_get_object_id'
#throwaway_df.select(throwaway_df.count).show()
throwaway_df.select(throwaway_df['count']).show()

+-----+
|count|
+-----+
|   10|
|   20|
|    5|
+-----+



To understand why that failed, you have to understand how the attribute-column syntax is implemented.

When you type `throwaway_df.count`, Python looks for an _existing_ attribute or method called `count` on the `throwaway_df` object. If it finds one, it uses it. Otherwise, it calls a special Python function (`__getattr__`), which defaults to throwing an exception. Spark has **overridden** `__getattr__` to look for a column on the DataFrame.

**This means you can only use the attribute (dot) syntax to refer to a column if the DataFrame does not _already_ have an attribute with the column's name.**

In the above example, there's already a `count()` method on the `DataFrame` class, so `throwaway_df.count` does not refer to our "count" column; instead, it refers to the `count()` _method_.

To avoid this problem, you can refer to the column using subscript notation: `throwaway_df['count']`. This syntax will _always_ work.
### (1a) Library Imports


We can import standard Python libraries ([modules](https://docs.python.org/2/tutorial/modules.html)) in the usual way.  An `import` statement will import the specified module.  In this lab, we will provide any imports that are necessary.

Let's import some of the libraries we'll need:

* `re`: The regular expression library
* `datetime`: Date and time functions
* `Test`: Our Databricks test helper library

In [3]:
import re
import datetime
#from databricks_test_helper import Test

## 零宽断言
用于查找在某些内容(但并不包括这些内容)之前或之后的东西，也就是说它们像\b,^,$那样用于指定一个位置，这个位置应该满足一定的条件(即断言)，因此它们也被称为零宽断言。

In [4]:
# Quick test of the regular expression library
m = re.search('(?<=abc)def','abcdef')
m.group(0)

'def'

In [5]:
# Quick test of the datetime library
print('This was last run on:{0}'.format(datetime.datetime.now()))

This was last run on:2016-10-01 07:57:40.553537


### (1b) Getting help

Remember: There are some useful Python built-ins for getting help.

You can use Python's [dir()](https://docs.python.org/2/library/functions.html?highlight=dir#dir) function to get a list of all the attributes (including methods) accessible through the `sqlContext` object.

In [7]:
# List sqlContext's attributes
# dir(sqlContext)

Alternatively, you can use Python's [help()](https://docs.python.org/2/library/functions.html?highlight=help#help) function to get an easier to read list of all the attributes, including examples, that the `sqlContext` object has.

In [9]:
# Use help to obtain more detailed information
# help(sqlContext)

## Part 2: Exploratory Data Analysis(探索性数据分析)

Let's begin looking at our data.  For this lab, we will use a data set from NASA Kennedy Space Center(肯尼迪航天中心) web server in Florida. The full data set is freely available at <http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html>, and it contains all HTTP requests for two months. We are using a subset that only contains several days' worth of requests.  The log file has already been downloaded for you.

>所谓探索性数据分析(Exploratory Data Analysis，以下简称EDA)，是指对已有的数据(特别是调查或观察得来的原始数据)在尽量少的先验假定下进行探索，通过作图、制表、方程拟合、计算特征量等手段探索数据的结构和规律的一种数据分析方法。特别是当我们对这些数据中的信息没有足够的经验，不知道该用何种传统统计方法进行分析时，探索性数据分析就会非常有效。

In [16]:
# Specify path to download log file
import sys
import os
log_file_path = 'datasets/' + os.path.join('apache.access.log.PROJECT')
log_file_path

'datasets/apache.access.log.PROJECT'

### (2a) Loading the log file

Now that we have the path to the file, let's load it into a DataFrame. We'll do this in steps. First, we'll use `sqlContext.read.text()` to read the text file. This will produce a DataFrame with a single string column called `value`.

In [18]:
base_df = sqlContext.read.text(log_file_path)
# Let's look at the schema
base_df.printSchema()

root
 |-- value: string (nullable = true)



Let's take a look at some of the data.

In [21]:
base_df.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


### (2b) Parsing the log file
If you're familiar with web servers at all, you'll recognize that this is in [Common Log Format](https://www.w3.org/Daemon/User/Config/Logging.html#common-logfile-format). The fields are:

_remotehost rfc931 authuser [date] "request" status bytes_

| field         | meaning                                                                |
| ------------- | ---------------------------------------------------------------------- |
| _remotehost_  | Remote hostname (or IP number if DNS hostname is not available). 主机的IP地址或者已解析的域名      |
| _rfc931_      | The remote logname of the user. We don't really care about this field. |
| _authuser_    | The username of the remote user, as authenticated by the HTTP server. 授权用户 |
| _[date]_      | The date and time of the request.    [日期/月份/年份:小时:分钟:秒钟时区]                                  |
| _"request"_   | The request, exactly as it came from the browser or client.在网站上通过何种方式获取了哪些信息            |
| _status_      | The HTTP status code the server sent back to the client.               |
| _bytes_       | The number of bytes (`Content-Length`) transferred to the client.      |


Next, we have to parse it into individual columns. We'll use the special built-in [regexp\_extract()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_extract)
function to do the parsing. This function matches a column against a regular expression with one or more [capture groups](http://regexone.com/lesson/capturing_groups) and allows you to extract one of the matched groups. We'll use one regular expression for each field we wish to extract.

If you can't read these regular expressions, don't worry. Trust us: They work. If you find regular expressions confusing (and they certainly _can_ be), and you want to learn more about them, start with the
[RegexOne web site](http://regexone.com/). You might also find [_Regular Expressions Cookbook_](http://shop.oreilly.com/product/0636920023630.do), by Jan Goyvaerts and Steven Levithan, to be helpful.

_Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems._ (attributed to Jamie Zawinski)